### Imports

In [ ]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn
import trainer_lib as tl
import torch_model_definitions as tmd

torch.manual_seed(310231551)
random.seed(3009231410)
np.random.seed(2909231846)
np_random_state = np.random.RandomState(131002)

### Load data

In [ ]:
df: pd.DataFrame = tl.load_country_wide_dataset('../data/country_data.csv')
df

### Define models

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, features=11, hidden_size=15, num_layers=2, dropout=0.0, hid_noise=0.0,
                 bidirectional=True, **kwargs):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.h_n_dim = 2 if bidirectional else 1
        self.num_layers = num_layers
        rec_drop = dropout if num_layers > 1 else 0.0
        self.lstm = nn.LSTM(input_size=features, hidden_size=self.hidden_size, num_layers=num_layers, batch_first=True,
                            bidirectional=bidirectional, dropout=rec_drop)
        # https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html
        self.fc = nn.Sequential(
            nn.Flatten(),
            tmd.GaussianNoise(hid_noise),
            nn.Dropout(dropout),
            nn.Linear(self.hidden_size * self.h_n_dim * self.num_layers, 1)
        )

    def forward(self, x):
        batch_size = x.shape[0]
        h_0 = torch.zeros(self.h_n_dim * self.num_layers, batch_size, self.hidden_size).requires_grad_().to(
            tl.TRAINER_LIB_DEVICE)
        c_0 = torch.zeros(self.h_n_dim * self.num_layers, batch_size, self.hidden_size).requires_grad_().to(
            tl.TRAINER_LIB_DEVICE)

        output, (h_n, c_n) = self.lstm(x, (h_0, c_0))
        h_n = torch.permute(h_n, (1, 0, 2))
        return self.fc(h_n) 


class GRUModel(nn.Module):
    def __init__(self, features=11, hidden_size=15, num_layers=2, dropout=0.0, hid_noise=0.0,
                 bidirectional=True, **kwargs):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size
        self.h_n_dim = 2 if bidirectional else 1
        self.num_layers = num_layers
        rec_drop = dropout if num_layers > 1 else 0.0
        self.gru = nn.GRU(input_size=features, hidden_size=self.hidden_size, num_layers=num_layers, batch_first=True,
                          bidirectional=bidirectional, dropout=rec_drop)
        # https://pytorch.org/docs/stable/generated/torch.nn.GRU.html
        self.fc = nn.Sequential(
            nn.Flatten(),
            tmd.GaussianNoise(hid_noise),
            nn.Dropout(dropout),
            nn.Linear(self.hidden_size * self.h_n_dim * self.num_layers, 1)
        )

    def forward(self, x):
        batch_size = x.shape[0]
        hidden = torch.zeros(self.h_n_dim * self.num_layers, batch_size, self.hidden_size).requires_grad_().to(
            tl.TRAINER_LIB_DEVICE)

        output, hidden = self.gru(x, hidden)
        hidden = torch.permute(hidden, (1, 0, 2))
        return self.fc(hidden)
    

class ConvNet(nn.Module):
    def __init__(self, channels=(32, 64), kernel_sizes=(12, 6), noise_sigma=0.0, dropout=0.0, **kwargs):
        super(ConvNet, self).__init__()
        self.seq_len = 24
        self.conv = nn.Sequential(
            nn.ZeroPad2d((kernel_sizes[0] // 2, 0, 0, 0)),
            nn.Conv1d(1, channels[0], kernel_sizes[0]),
            nn.ReLU(),
            nn.MaxPool1d(2, padding=1),
            nn.ZeroPad2d((kernel_sizes[1] // 2, 0, 0, 0)),
            nn.Conv1d(channels[0], channels[1], kernel_sizes[1]),
            nn.ReLU(),
            nn.MaxPool1d(2, padding=0),
        )
        out = self.conv(torch.randn(1, 1, self.seq_len)).shape[-1]
        self.fc = nn.Sequential(
            tmd.GaussianNoise(noise_sigma),
            nn.Flatten(1, -1),
            nn.Dropout(dropout),
            nn.Linear(channels[1] * out, 1),
        )

    def forward(self, x):
        x = x.reshape(-1, 1, self.seq_len)
        x = self.conv(x)
        x = self.fc(x)
        return x
    
    
class TCN(nn.Module):
    def __init__(self, num_channels=(24,) * 2, kernel_size=3, dropout=0.5, hid_noise=0.0, **kwargs):
        super(TCN, self).__init__()
        self.seq_len = 24
        self.pred_len = 1
        self.num_channels = num_channels
        self.kernel_size = kernel_size
        self.dropout = dropout
        self.tcn = tmd.TemporalConvNet(1, num_channels, kernel_size=kernel_size, dropout=dropout)
        self.hid_noise = tmd.GaussianNoise(hid_noise)
        self.fc = nn.Linear(num_channels[-1], 1)

    def forward(self, x):
        x = x.reshape(-1, 1, self.seq_len)
        x = self.tcn(x)
        x = self.hid_noise(x)
        return self.fc(x[:, :, -1])
    
    

# Individual testing

    ### precipitation

In [ ]:
X = df['prec'].to_numpy(dtype=np.float32).reshape(-1, 1)
y = X.copy()

grid = tl.Grid({
    'epochs': [1000],  # we use early stopping, so this is just a high number
    'lr': [0.0005],
    'model': [ConvNet],
    'kernel_sizes': [(12, 6), (10, 10), (6, 12)],
    'channels': [(8, 16), (16, 32)],
    'noise_sigma': [0.05],
    'dropout': [0.5],
    'batch_size': [2048],
})

wrapper: tl.RECOneModelTSWrapper = tl.RECOneModelTSWrapper(ConvNet(), 24, 3)
b_p, b_s = wrapper.grid_search(X, y, grid, verbose=2)
print(f"\nBest params: {b_p}\nBest score: {b_s}")

In [ ]:
X = df['prec'].to_numpy(dtype=np.float32).reshape(-1, 1)
y = X.copy()

grid = tl.Grid({
    'epochs': [1000],  # we use early stopping, so this is just a high number
    'lr': [0.001],
    'model': [TCN],
    'kernel_size': [3, 5],
    'num_channels': [(16, 32), (32, 32)],
    'noise_sigma': [0.05],
    'dropout': [0.5],
    'batch_size': [2048],
})

wrapper: tl.RECOneModelTSWrapper = tl.RECOneModelTSWrapper(TCN(), 24, 3)
b_p, b_s = wrapper.grid_search(X, y, grid, verbose=2)
print(f"\nBest params: {b_p}\nBest score: {b_s}")

In [ ]:
X = df['prec'].to_numpy(dtype=np.float32).reshape(-1, 1)
y = X.copy()

grid = tl.Grid({
    'epochs': [1000],  # we use early stopping, so this is just a high number
    'lr': [0.001],
    'model': [LSTMModel, GRUModel],
    'features': [1],
    'hidden_size': [15, 20],
    'num_layers': [2],
    'bidirectional': [True],
    'hid_noise': [0.05],
    'dropout': [0.5],
    'batch_size': [2048],
})

wrapper: tl.RECOneModelTSWrapper = tl.RECOneModelTSWrapper(GRUModel(), 24, 3)
b_p, b_s = wrapper.grid_search(X, y, grid, verbose=2)
print(f"\nBest params: {b_p}\nBest score: {b_s}")

### global radiation

In [ ]:
X = df['grad'].to_numpy(dtype=np.float32).reshape(-1, 1)
y = X.copy()

grid = tl.Grid({
    'epochs': [1000],  # we use early stopping, so this is just a high number
    'lr': [0.0005],
    'model': [ConvNet],
    'kernel_sizes': [(12, 6), (10, 10), (6, 12)],
    'channels': [(8, 16), (16, 32)],
    'noise_sigma': [0.05],
    'dropout': [0.5],
    'batch_size': [2048],
})

wrapper: tl.RECOneModelTSWrapper = tl.RECOneModelTSWrapper(ConvNet(), 24, 3)
b_p, b_s = wrapper.grid_search(X, y, grid, verbose=2)
print(f"\nBest params: {b_p}\nBest score: {b_s}")

In [ ]:
X = df['grad'].to_numpy(dtype=np.float32).reshape(-1, 1)
y = X.copy()

grid = tl.Grid({
    'epochs': [1000],  # we use early stopping, so this is just a high number
    'lr': [0.001],
    'model': [TCN],
    'kernel_size': [3, 5],
    'num_channels': [(16, 32), (32, 32)],
    'noise_sigma': [0.05],
    'dropout': [0.5],
    'batch_size': [2048],
})

wrapper: tl.RECOneModelTSWrapper = tl.RECOneModelTSWrapper(TCN(), 24, 3)
b_p, b_s = wrapper.grid_search(X, y, grid, verbose=2)
print(f"\nBest params: {b_p}\nBest score: {b_s}")

In [ ]:
X = df['grad'].to_numpy(dtype=np.float32).reshape(-1, 1)
y = X.copy()

grid = tl.Grid({
    'epochs': [1000],  # we use early stopping, so this is just a high number
    'lr': [0.001],
    'model': [LSTMModel, GRUModel],
    'features': [1],
    'hidden_size': [10, 15, 20],
    'num_layers': [2],
    'bidirectional': [True],
    'hid_noise': [0.05],
    'dropout': [0.5],
    'batch_size': [2048],
})

wrapper: tl.RECOneModelTSWrapper = tl.RECOneModelTSWrapper(GRUModel(), 24, 3)
b_p, b_s = wrapper.grid_search(X, y, grid, verbose=2)
print(f"\nBest params: {b_p}\nBest score: {b_s}")

### el_load non-recursive

In [ ]:
X = df.to_numpy(dtype=np.float32)
y = df['el_load'].to_numpy(dtype=np.float32)

grid = tl.Grid({
    'epochs': [1000],  # we use early stopping, so this is just a high number
    'lr': [0.001],
    'model': [LSTMModel, GRUModel],
    'hidden_size': [15, 25, 35],
    'num_layers': [2, 3],
    'bidirectional': [True],
    'batch_size': [2048],
    'dropout': [0.5],
    'hid_noise': [0.05],
})

wrapper: tl.MIMOTSWrapper = tl.MIMOTSWrapper(LSTMModel(), seq_len=24, pred_len=1)
b_p, b_s = wrapper.grid_search(X, y, grid, verbose=3)
print(f"\nBest params: {b_p}\nBest score: {b_s}")

No major differences here, in the first run GRU with 25 hidden size and 2 layers were the best

In [ ]:
X = df.to_numpy(dtype=np.float32)
y = df['el_load'].to_numpy(dtype=np.float32)

grid = tl.Grid({
    'epochs': [1000],  # we use early stopping, so this is just a high number
    'lr': [0.001],
    'model': [LSTMModel, GRUModel],
    'hidden_size': [10, 15],
    'num_layers': [2],
    'bidirectional': [True],
    'batch_size': [2048],
    'dropout': [0.5],
    'hid_noise': [0.05],
})

wrapper: tl.MIMOTSWrapper = tl.MIMOTSWrapper(LSTMModel(), seq_len=24, pred_len=1)
b_p, b_s = wrapper.grid_search(X, y, grid, verbose=3)
print(f"\nBest params: {b_p}\nBest score: {b_s}")

let's try a single layer

In [ ]:
X = df.to_numpy(dtype=np.float32)
y = df['el_load'].to_numpy(dtype=np.float32)

grid = tl.Grid({
    'epochs': [1000],  # we use early stopping, so this is just a high number
    'lr': [0.001],
    'model': [GRUModel],
    'hidden_size': [25, 30, 35],
    'num_layers': [1],
    'bidirectional': [True],
    'batch_size': [2048],
    'dropout': [0.5],
    'hid_noise': [0.05],
    'es_p': [20],
})

wrapper: tl.MIMOTSWrapper = tl.MIMOTSWrapper(GRUModel(), seq_len=24, pred_len=1)
b_p, b_s = wrapper.grid_search(X, y, grid, verbose=3)
print(f"\nBest params: {b_p}\nBest score: {b_s}")

In [ ]:
X = df.to_numpy(dtype=np.float32)
y = df['el_load'].to_numpy(dtype=np.float32)

grid = tl.Grid({
    'epochs': [1000],  # we use early stopping, so this is just a high number
    'lr': [0.001],
    'model': [GRUModel],
    'hidden_size': [40, 45],
    'num_layers': [1],
    'bidirectional': [True],
    'batch_size': [2048],
    'dropout': [0.5],
    'hid_noise': [0.05],
    'es_p': [20],
})

wrapper: tl.MIMOTSWrapper = tl.MIMOTSWrapper(GRUModel(), seq_len=24, pred_len=1)
b_p, b_s = wrapper.grid_search(X, y, grid, verbose=3)
print(f"\nBest params: {b_p}\nBest score: {b_s}")

### putting it all together

In [ ]:
class MultiModelRec(nn.Module):
    def __init__(self, features=11, pred_len=3, hidden_size=15, num_layers=2, dropout=0.0,
                 hid_noise=0.0, bidirectional=True,  **kwargs):
        super(MultiModelRec, self).__init__()
        self.out_features = 3
        self.pred_len = pred_len
        
        self.gru = GRUModel(features, hidden_size, num_layers, dropout, hid_noise, bidirectional)
        self.tcn = TCN((32,) * 2, kernel_size=5, dropout=dropout, hid_noise=hid_noise)
        self.conv = ConvNet((16, 32), (6, 12), 0.5, 0.05)


    def forward(self, x, y, teacher_forcing=0.0):
        batch_size = x.shape[0]

        if y.shape[2] != self.gru.gru.input_size:
            pre_calc = torch.concat((
                torch.zeros(batch_size, self.pred_len, self.out_features).to(tl.TRAINER_LIB_DEVICE),
                y), dim=2)
            teacher_forcing = 0.0
        else:
            pre_calc = y

        output = torch.zeros(batch_size, self.pred_len).to(tl.TRAINER_LIB_DEVICE)

        for i in range(self.pred_len):
            out = torch.concat((
                self.gru(x),
                self.tcn(x[:, :, 1]),
                self.conv(x[:, :, 2])
            ), dim=1)

            output[:, i] = out[:, 0]
            
            x = torch.cat((x[:, 1:], pre_calc[:, i].unsqueeze(1)), dim=1)
            for j in range(self.out_features):  # roll teacher forcing for each feature
                if torch.rand(1) > teacher_forcing:
                    x[:, -1, j] = out[:, j]

        return output


X = df.to_numpy(dtype=np.float32)
y = X.copy()

grid = tl.Grid({
    'epochs': [1000],  # we use early stopping, so this is just a high number
    'lr': [0.001],
    'model': [MultiModelRec],
    'hidden_size': [25, 30, 35],
    'num_layers': [2],
    'bidirectional': [True],
    'batch_size': [1024],
    'dropout': [0.5],
    'hid_noise': [0.05],
    'es_p': [20],
})

wrap: tl.RECMultiModelTSWrapper = tl.RECMultiModelTSWrapper(MultiModelRec(), 24, 3, 3, teacher_forcing_decay=0.01)
b_p, b_s = wrap.grid_search(X, y, grid, verbose=4)
print(f"\nBest params: {b_p}\nBest score: {b_s}")

In [ ]:
X = df.to_numpy(dtype=np.float32)
y = X.copy()

grid = tl.Grid({
    'epochs': [1000],  # we use early stopping, so this is just a high number
    'lr': [0.001],
    'model': [MultiModelRec],
    'hidden_size': [55, 50, 45],
    'num_layers': [1],
    'bidirectional': [True],
    'batch_size': [1024],
    'dropout': [0.5],
    'hid_noise': [0.05],
    'es_p': [25],
})

wrap: tl.RECMultiModelTSWrapper = tl.RECMultiModelTSWrapper(MultiModelRec(), 24, 3, 3, teacher_forcing_decay=0.01)
b_p, b_s = wrap.grid_search(X, y, grid, verbose=4)
print(f"\nBest params: {b_p}\nBest score: {b_s}")